# Molecule Feature & Distance Matrices

Import all the necessary packages:

In [1]:
import numpy as np
import pandas as pd
import pyrfume
from ast import literal_eval
from pyrfume.features import smiles_to_mordred, smiles_to_morgan
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import pdist, squareform

Load the identifiers.csv and molecules.csv as DataFrames:

In [2]:
identifiers = pd.read_csv('identifiers.csv')
molecules = pd.read_csv('molecules.csv')

Gather all SMILES of molecules as a list:

In [3]:
smiles = molecules['IsomericSMILES'].tolist()

Obtain all mordred features from SMILES:

In [4]:
mordred_features = smiles_to_mordred(smiles)

  0%|          | 0/20 [00:00<?, ?it/s]


Computing Mordred features...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 37.04it/s]


There are 20 molecules and 1826 features


In [5]:
mordred_features.head()

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
C1=CC=C(C=C1)C=O,5.656854,5.42766,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,2.969338,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.0
CCCC(=O)O,3.754314,4.057055,1,0,6.155367,1.902113,3.804226,6.155367,1.025895,2.5951,...,7.131699,29.439488,88.052429,6.289459,32,3,20.0,18.0,3.611111,1.583333
CCC(=O)O,3.047207,3.305183,1,0,5.226252,1.847759,3.695518,5.226252,1.04525,2.408576,...,6.834109,27.25413,74.036779,6.730616,18,2,16.0,14.0,3.361111,1.333333
CC(C)C(=O)O,3.932653,4.244375,1,0,6.0,2.0,4.0,6.0,1.0,2.610845,...,7.626083,30.69869,88.052429,6.289459,29,4,22.0,21.0,4.222222,1.444444
CCC(C)CC(=O)O,5.277917,5.655215,1,0,8.565187,2.042079,4.084158,8.565187,1.070648,2.899769,...,7.8842,34.080836,116.08373,5.804186,71,6,30.0,29.0,4.722222,2.0


Merge CIDs with DataFrame of features:

In [6]:
features = pd.merge(molecules, mordred_features, left_on='IsomericSMILES', right_index=True)
features = features.drop(['MolecularWeight', 'IsomericSMILES', 'IUPACName', 'name'], axis=1)
features.head()

,CID,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,240,5.656854,5.42766,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.0
1,264,3.754314,4.057055,1,0,6.155367,1.902113,3.804226,6.155367,1.025895,...,7.131699,29.439488,88.052429,6.289459,32,3,20.0,18.0,3.611111,1.583333
2,1032,3.047207,3.305183,1,0,5.226252,1.847759,3.695518,5.226252,1.04525,...,6.834109,27.25413,74.036779,6.730616,18,2,16.0,14.0,3.361111,1.333333
3,6590,3.932653,4.244375,1,0,6.0,2.0,4.0,6.0,1.0,...,7.626083,30.69869,88.052429,6.289459,29,4,22.0,21.0,4.222222,1.444444
4,7755,5.277917,5.655215,1,0,8.565187,2.042079,4.084158,8.565187,1.070648,...,7.8842,34.080836,116.08373,5.804186,71,6,30.0,29.0,4.722222,2.0


Make values in CIDs column in identifiers be list type and iterate through each one. If a stimulus contains multiple CIDs, then the feature vectors for the CIDs will averaged and a new row for this mixture stimulus is added to the features matrix:

In [7]:
identifiers['CIDs'] = identifiers['CIDs'].apply(literal_eval)

In [8]:
temp = pd.DataFrame()

for i, row in identifiers[['CIDs', 'conc']].iterrows():
    cids = row['CIDs']
    conc = row['conc']
    if len(cids) == 2 and conc == 1:
        cid1 = pd.DataFrame(features.loc[features['CID'] == cids[0]]).drop('CID', axis=1)
        cid2 = pd.DataFrame(features.loc[features['CID'] == cids[1]]).drop('CID', axis=1)
        combined = pd.concat([cid1, cid2]).mean()
        combined['CID'] = cids
        temp = pd.concat([temp, combined], axis=1)

temp = temp.T.reset_index(drop=True)
temp

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,CID
0,5.357996,5.432081,0.0,0.0,9.916776,2.062411,4.124823,9.916776,1.239597,2.928929,...,34.310093,110.073165,6.380493,70.0,6.5,31.0,31.5,3.361111,2.083333,"[240, 7802]"
1,5.629974,5.994146,1.0,0.0,9.02506,2.216046,4.358876,9.02506,1.128132,2.954465,...,42.352163,115.075905,6.070651,62.0,7.5,35.0,37.5,4.322917,1.868056,"[11684, 18840]"
2,6.65983,6.585253,0.5,0.0,10.374839,2.298271,4.596542,10.374839,1.144549,3.100962,...,38.267697,126.104465,5.519886,86.5,9.5,44.0,50.5,4.434028,2.013889,"[7755, 82227]"
3,6.453131,6.283695,0.5,0.0,11.303566,2.182425,4.291634,11.303566,1.251294,3.082017,...,43.417054,123.062797,6.836822,97.5,7.5,39.0,41.5,3.166667,2.180556,"[18840, 637511]"
4,5.113832,5.399176,0.5,0.0,9.085916,1.96945,3.9389,9.085916,1.13574,2.888253,...,33.231456,115.094097,5.495377,77.5,5.5,28.0,26.5,4.111111,2.125,"[7802, 8892]"
5,4.979058,5.298847,1.0,0.0,7.696807,2.021039,4.042079,7.696807,1.023069,2.832439,...,33.146371,109.075905,5.904096,59.5,5.0,28.0,26.5,4.597222,1.833333,"[7755, 10430]"
6,8.439677,7.82508,0.5,0.5,13.828832,2.159652,4.258472,13.828832,1.169026,3.264222,...,46.919895,160.104996,6.194452,236.5,11.0,52.0,55.5,4.708333,2.638889,"[12587, 17750155]"
7,7.269627,6.68739,0.5,0.5,12.20589,2.083531,4.106231,12.20589,1.162143,3.018896,...,43.64138,139.081521,6.657667,208.5,9.5,45.0,48.5,4.027778,2.347222,"[1032, 17750155]"
8,6.55044,6.475896,0.0,0.0,10.796875,2.271754,4.543507,10.796875,1.197303,3.095338,...,37.913555,125.114833,5.211076,89.0,9.5,43.0,49.5,4.128472,2.097222,"[7802, 82227]"
9,4.814974,5.049282,1.0,0.0,8.24499,1.940854,3.881707,8.24499,1.099618,2.820106,...,32.21325,109.075905,5.904096,65.5,4.5,26.0,24.0,3.986111,1.958333,"[7991, 8892]"


Create a new DataFrame copy of features, with the CID column as a list of CIDs, then concatenate the temporary DataFrame of averaged features from mixtures:

In [9]:
all_features = features.copy()
all_features['CID'] = all_features[['CID']].apply(lambda x: [x['CID']], axis=1)

In [10]:
all_features = pd.concat([all_features, temp]).reset_index(drop=True)
all_features

,CID,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,[240],5.656854,5.42766,0,0,10.424292,2.135779,4.271558,10.424292,1.303037,...,8.298291,35.247635,106.041865,7.574419,64,7,34.0,36.0,2.611111,2.0
1,[264],3.754314,4.057055,1,0,6.155367,1.902113,3.804226,6.155367,1.025895,...,7.131699,29.439488,88.052429,6.289459,32,3,20.0,18.0,3.611111,1.583333
2,[1032],3.047207,3.305183,1,0,5.226252,1.847759,3.695518,5.226252,1.04525,...,6.834109,27.25413,74.036779,6.730616,18,2,16.0,14.0,3.361111,1.333333
3,[6590],3.932653,4.244375,1,0,6.0,2.0,4.0,6.0,1.0,...,7.626083,30.69869,88.052429,6.289459,29,4,22.0,21.0,4.222222,1.444444
4,[7755],5.277917,5.655215,1,0,8.565187,2.042079,4.084158,8.565187,1.070648,...,7.8842,34.080836,116.08373,5.804186,71,6,30.0,29.0,4.722222,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,"[14057, 82227]",6.843513,6.69461,0.5,0.0,9.784568,2.351712,4.703424,9.784568,1.070765,...,9.229582,38.92378,126.104465,5.519886,84.0,9.0,46.0,52.0,4.784722,1.868056
68,"[264, 8034]",4.57081,4.818809,0.5,0.0,7.313752,1.951057,3.902113,7.313752,1.042456,...,7.435093,31.625324,101.078447,5.738013,53.0,4.0,25.0,23.0,4.166667,1.75
69,"[8314, 10430]",4.605285,4.973283,1.0,0.0,7.245708,2.02644,4.052881,7.245708,1.035101,...,7.77341,32.450329,102.06808,6.004005,47.0,5.0,26.0,25.0,4.472222,1.722222
70,"[7802, 12587]",5.223222,5.508533,0.5,0.0,8.940698,1.994522,3.989044,8.940698,1.117587,...,7.680821,33.591856,115.094097,5.495377,75.0,5.5,29.0,27.5,4.416667,2.041667


Reset the type of the CID columns for features and identifiers to strings so that they can be easily joined:

In [11]:
all_features['CID'] = all_features[['CID']].apply(lambda x: str(x['CID']), axis=1)
identifiers['CIDs'] = identifiers[['CIDs']].apply(lambda x: str(x['CIDs']), axis=1)
identifiers = identifiers[identifiers['conc'] != 0]
identifiers

,stimulus,CIDs,conc
0,1,"[240, 7802]",1
2,3,"[11684, 18840]",1
3,4,[18840],1
5,6,[1032],1
7,8,"[7755, 82227]",1
...,...,...,...
135,136,"[14057, 82227]",1
136,137,"[264, 8034]",1
138,139,"[8314, 10430]",1
139,140,"[7802, 12587]",1


Bring the stimuli identifiers into DataFrame and set the stimulus as the index:

In [12]:
stimuli_features = pd.merge(all_features, identifiers, left_on='CID', right_on='CIDs')
stimuli_features = stimuli_features.drop(['CID', 'CIDs', 'conc'], axis=1)
stimuli_features = stimuli_features.set_index('stimulus')

Normalize all columns of the features matrix, dropping columns that contain NaNs:

In [13]:
scaler = MinMaxScaler()
stimuli_features[stimuli_features.columns] = scaler.fit_transform(stimuli_features[stimuli_features.columns])

/home/dswalla/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:461: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/dswalla/.local/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:462: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [14]:
stimuli_features = stimuli_features.dropna(axis='columns')
stimuli_features

,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,SpDiam_A,SpAD_A,SpMAD_A,LogEE_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
stimulus,,,,,,,,,,,,,,,,,,,,,
70,0.309023,0.313771,0.0,0.0,0.372372,0.407554,0.407554,0.372372,1.000000,0.459400,...,0.451076,0.243894,0.246024,0.922029,0.120735,0.333333,0.310345,0.318841,0.000000,0.328767
119,0.083732,0.111151,1.0,0.0,0.066559,0.076912,0.076912,0.066559,0.270594,0.152808,...,0.091680,0.066679,0.107739,0.425863,0.036745,0.066667,0.068966,0.057971,0.355556,0.123288
6,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.321536,0.000000,...,0.000000,0.000000,0.000000,0.596209,0.000000,0.000000,0.000000,0.000000,0.266667,0.000000
56,0.104851,0.138843,1.0,0.0,0.055429,0.215424,0.215424,0.055429,0.202443,0.165708,...,0.243986,0.105099,0.107739,0.425863,0.028871,0.133333,0.103448,0.101449,0.572840,0.054795
118,0.264151,0.347411,1.0,0.0,0.239191,0.274966,0.274966,0.239191,0.388381,0.402407,...,0.323506,0.208293,0.323216,0.238483,0.139108,0.266667,0.241379,0.217391,0.750617,0.328767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0.449541,0.501067,0.5,0.0,0.326544,0.713103,0.713103,0.326544,0.388688,0.576518,...,0.737983,0.356059,0.400245,0.128705,0.173228,0.466667,0.517241,0.550725,0.772840,0.263699
137,0.180418,0.223763,0.5,0.0,0.149542,0.146168,0.146168,0.149542,0.314181,0.277386,...,0.185148,0.133372,0.207870,0.212932,0.091864,0.133333,0.155172,0.130435,0.553086,0.205479
139,0.184501,0.246599,1.0,0.0,0.144668,0.252838,0.252838,0.144668,0.294825,0.292582,...,0.289374,0.158544,0.215477,0.315640,0.076115,0.200000,0.172414,0.159420,0.661728,0.191781


Calculate the euclidean distance between all pairs of stimuli:

In [15]:
distance_matrix = pdist(stimuli_features, 'euclidean')

Create functions to easily lookup feature distance (euclidian distance) between two stimuli:

In [16]:
def get_pdist_index(stim1, stim2, features):
    i = features.index.get_loc(stim1)
    j = features.index.get_loc(stim2)
    m = features.shape[0]
    if i > j:
        index = m * j + i - ((j + 2) * (j + 1)) // 2
    else:    
        index = m * i + j - ((i + 2) * (i + 1)) // 2
    return index

def feature_distance_from_stimuli(stim1, stim2, features, distance_matrix):
    i = get_pdist_index(stim1, stim2, features)
    return distance_matrix[i]

Test out a pair of stimuli:

In [17]:
dist = feature_distance_from_stimuli(70, 6, stimuli_features, distance_matrix)
print(dist)

14.710942117834673


## Conversion to squareform and write out to file

In [18]:
sqr = pd.DataFrame(squareform(distance_matrix))
sqr.head()

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,0.000000,12.474607,14.710942,13.245094,12.371401,12.918498,11.958554,13.430166,12.949262,11.836654,...,13.774317,12.926530,7.117870,6.237304,12.508600,14.314662,11.666097,11.901443,12.155128,10.240167
1,12.474607,0.000000,7.347430,7.351617,6.671245,11.082668,4.561606,11.295092,10.821586,5.586102,...,7.529168,7.859074,8.038837,6.237304,6.517481,12.930942,5.647546,4.224088,8.537239,9.918832
2,14.710942,7.347430,0.000000,5.761759,11.310690,14.567134,9.076145,14.429674,14.426078,8.276055,...,12.380241,8.484698,12.221782,9.812967,10.312066,16.531130,9.960188,8.927435,12.357051,10.897982
3,13.245094,7.351617,5.761759,0.000000,9.050677,12.565252,7.944051,11.831288,12.460546,5.550132,...,9.681710,3.984486,10.201615,8.708354,6.837714,13.304253,8.069576,6.386328,9.998956,10.209425
4,12.371401,6.671245,11.310690,9.050677,0.000000,8.547679,4.458944,7.872903,8.186836,5.870663,...,6.646069,7.589858,6.813722,7.737866,5.215820,9.479585,4.620528,4.721606,5.084933,9.871111


In [19]:
sqr.index = stimuli_features.index
sqr.columns = stimuli_features.index
sqr.head()

stimulus,70,119,6,56,118,103,102,110,51,98,...,122,126,127,131,133,136,137,139,140,142
stimulus,,,,,,,,,,,,,,,,,,,,,
70,0.000000,12.474607,14.710942,13.245094,12.371401,12.918498,11.958554,13.430166,12.949262,11.836654,...,13.774317,12.926530,7.117870,6.237304,12.508600,14.314662,11.666097,11.901443,12.155128,10.240167
119,12.474607,0.000000,7.347430,7.351617,6.671245,11.082668,4.561606,11.295092,10.821586,5.586102,...,7.529168,7.859074,8.038837,6.237304,6.517481,12.930942,5.647546,4.224088,8.537239,9.918832
6,14.710942,7.347430,0.000000,5.761759,11.310690,14.567134,9.076145,14.429674,14.426078,8.276055,...,12.380241,8.484698,12.221782,9.812967,10.312066,16.531130,9.960188,8.927435,12.357051,10.897982
56,13.245094,7.351617,5.761759,0.000000,9.050677,12.565252,7.944051,11.831288,12.460546,5.550132,...,9.681710,3.984486,10.201615,8.708354,6.837714,13.304253,8.069576,6.386328,9.998956,10.209425
118,12.371401,6.671245,11.310690,9.050677,0.000000,8.547679,4.458944,7.872903,8.186836,5.870663,...,6.646069,7.589858,6.813722,7.737866,5.215820,9.479585,4.620528,4.721606,5.084933,9.871111


In [20]:
sqr.to_csv('stimuli_feature_distance.csv')

In [23]:
sqr.loc[(119, 70)]

12.474607257166252

## Manual validation of pdist

In [18]:
def calculate_pdist_manual(stim1, stim2):
    dist_ex = np.linalg.norm(stimuli_features.loc[stim1].values - stimuli_features.loc[stim2].values)
    print(dist_ex)

In [19]:
calculate_pdist_manual(70, 6)

14.710942117834673
